In [7]:
from utils import read_json_file, write_json_file, read_json_file_or_empty_list
from utils import large_num_short_format
from dotenv import load_dotenv
from typing import  List, Dict
from utils import print_progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  math
import os
import pickle
import web3

load_dotenv()

True

In [8]:
# Load AUTO charts and convert them to daily 
from download_chart_data import ChartInterval
from download_chart_data import load_chart_by_interval, chart_data_to_df
from download_candles import load_candles_by_interval, CandleInterval


daily_candles = load_candles_by_interval(CandleInterval.DAILY)
print("\nDaily candles count: {}".format(len(daily_candles)))
charts = load_chart_by_interval(ChartInterval.FULL)
print("\nCharts count: {}".format(len(charts)))

 Loading candles_daily idx: 10425 / 10426, 100.0%       
Daily candles count: 9796
 Loading chart_full idx: 10424 / 10425, 100.0%       
Charts count: 9887


In [9]:
# Transform charts to data frame
df_charts: Dict[str, pd.DataFrame] = dict()
total = len(charts)

for idx, key in enumerate(charts):
    chart = charts[key]
    df_charts[key] = chart_data_to_df(chart)
    print_progress("Transforming charts to dataframe ", idx, total, True)

 Transforming charts to dataframe  idx: 9886 / 9887, 100.0%       

In [10]:
from download_candles import daily_candle_data_to_df
# Find pumps for candles
from download_chart_data import chart_data_to_df
from download_coin import get_address_and_decimals
from utils import print_progress, year_month_str, year_month_day_str
import pandas as pd
import numpy as np

import  math


def is_pump(
    open: float, high: float, pump_ratio: float, min_volume: float, volume: float
) -> bool:
    if math.isnan(open) or math.isnan(high) or math.isnan(volume):
        return False
    return high >= open * pump_ratio and volume >= min_volume


def daily_candle_pumps_info(
    candle_data: List[List[any]], pump_ratio: float, min_volume: float, 
    chart_df: pd.DataFrame, shift_volume: bool = False
) -> pd.DataFrame:
    df = daily_candle_data_to_df(candle_data)    
    if shift_volume:
        chart_df = chart_df.shift(1, axis = 0)
    df = df.join(chart_df)
    df = df.drop(['price', 'market_cap'], axis=1)    
    df['Ymd'] = list(map(lambda x: year_month_day_str(x), df.index))
    df['pump'] = (df['high'] >= df['open'] * pump_ratio) & (df['volume'] > min_volume)
    return df.loc[df['pump'] == True]


pump_ratio = 2.0
min_volume = 500000
df_candles_pumps: Dict[str, pd.DataFrame] = dict()


# for each chart
for idx, key in enumerate(daily_candles):
    candles = daily_candles[key]
    print_progress("Searching for pumps", idx, len(daily_candles), True)
    if key not in df_charts: 
        continue    
    df_chart = df_charts[key]
    df_info = daily_candle_pumps_info(candles, pump_ratio, min_volume, df_chart)
    if not df_info.empty:
        df_candles_pumps[key] = df_info
        

print("") 
print("Coins that pumped", len(df_candles_pumps.keys()))

 Searching for pumps idx: 9795 / 9796, 100.0%       
Coins that pumped 424


In [11]:
# Filter out only ETH  pumps

eth_df_candles_pumps = dict()

for key, df in df_candles_pumps.items():
    address, decimals = get_address_and_decimals(key)
    if address is None or decimals is None:
        continue
    
    eth_df_candles_pumps[key] = df
        
print(sorted(eth_df_candles_pumps.keys()))
print("Eth coins pumps", len(eth_df_candles_pumps.keys()))

tmp_id = 'youwho'
tmp_df = eth_df_candles_pumps[tmp_id]
print(tmp_id, tmp_df.shape)
for i in range(len(tmp_df.index)):
    print(year_month_day_str(tmp_df.index[i]))
# jun(6) jul(7) aug(8) sep(9)

['0xcoco', '4chan', 'aardvark', 'acent', 'adreward', 'agility', 'alphai', 'amber-phantom-butterfly', 'anchor-protocol', 'apes-go-bananas', 'apiens', 'archimedes', 'arti-project', 'asap-sniper-bot', 'assangedao', 'astra-protocol-2', 'augur', 'axe-cap', 'axl-inu', 'baby-shiba-inu-erc', 'babypepe', 'barbiecrashbandicootrfk88', 'base-protocol', 'bear-inu', 'bedlingtonterriertoken', 'beep-coin', 'ben-2', 'ben-s-finale', 'biaocoin', 'bifi', 'big-time', 'billy-token', 'bitrock', 'blackpearl-chain', 'block-browser', 'blockbank', 'blockremit', 'blocktools', 'bob-token', 'bobo', 'bobo-coin', 'bogdanoff', 'boltbot', 'botxcoin', 'bridge-bot', 'broccoli', 'brz', 'bull-market', 'bullet-game', 'capybara-token', 'carecoin', 'caroline', 'celsius-degree-token', 'centrality', 'chad-coin', 'cindicator', 'clips', 'coinbot', 'coinhub', 'companionbot', 'connext', 'cope-coin', 'crypto-x', 'cryptopawcoin', 'cult-dao', 'cyberconnect', 'dawn-protocol', 'defidollar-dao', 'degen-zoo', 'delysium', 'dinger-token', '

In [1]:
from download_candles import daily_candle_data_to_df
from download_chart_data import chart_data_to_df
from utils import print_progress, year_month_str, year_month_day_str
from utils import read_json_file, write_json_file, read_json_file_or_empty_list
from utils import large_num_short_format
from dotenv import load_dotenv
from typing import  List, Dict
from utils import print_progress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import web3
import math
from download_coin import get_address_and_decimals


pickled_path = './results/pickled/'

# store
# for key in eth_df_candles_pumps.keys():
    # eth_df_candles_pumps[key].to_pickle(pickled_path + key)
    
# load
eth_df_candles_pumps = dict()
for file in os.listdir(pickled_path):
    eth_df_candles_pumps[file] = pd.read_pickle(pickled_path + file)

In [2]:
print(sorted(eth_df_candles_pumps.keys()))

coin_id = "spurdo" # wojak
df = eth_df_candles_pumps[coin_id]
print(coin_id)
for i in range(len(df.index)):
    print(year_month_day_str(df.index[i]))
timestamp = df.index[0]

['0xcoco', '4chan', 'aardvark', 'acent', 'adreward', 'agility', 'alphai', 'amber-phantom-butterfly', 'anchor-protocol', 'apes-go-bananas', 'apiens', 'archimedes', 'arti-project', 'asap-sniper-bot', 'assangedao', 'astra-protocol-2', 'augur', 'axe-cap', 'axl-inu', 'baby-shiba-inu-erc', 'babypepe', 'barbiecrashbandicootrfk88', 'base-protocol', 'bear-inu', 'bedlingtonterriertoken', 'beep-coin', 'ben-2', 'ben-s-finale', 'biaocoin', 'bifi', 'big-time', 'billy-token', 'bitrock', 'blackpearl-chain', 'block-browser', 'blockbank', 'blockremit', 'blocktools', 'bob-token', 'bobo', 'bobo-coin', 'bogdanoff', 'boltbot', 'botxcoin', 'bridge-bot', 'broccoli', 'brz', 'bull-market', 'bullet-game', 'capybara-token', 'carecoin', 'caroline', 'celsius-degree-token', 'centrality', 'chad-coin', 'cindicator', 'clips', 'coinbot', 'coinhub', 'companionbot', 'connext', 'cope-coin', 'crypto-x', 'cryptopawcoin', 'cult-dao', 'cyberconnect', 'dawn-protocol', 'defidollar-dao', 'degen-zoo', 'delysium', 'dinger-token', '

In [ ]:
from web3_utils import format_decimals, get_block_near
from utils import read_str_file, year_month_day_str, ranges_list
from web3 import Web3, AsyncWeb3
import web3
from multiprocessing import Pool, cpu_count
import datetime
import time


provider_url = 'http://127.0.0.1:8545'

def new_provider(url: str, timeout: int = 120) -> web3.Web3:
    return Web3(Web3.HTTPProvider(url, request_kwargs={'timeout': timeout}))
    
provider = new_provider(provider_url)

def block_num_for_timestamp(timestamp: int, provider: web3.Web3) -> int:
    return get_block_near(timestamp, provider)
    

def log_to_list(log: any, decimals: int) -> List[any]:
    val = log.args['value']
    return [
        provider.to_hex(log['transactionHash']), # tx_hash
        log['transactionIndex'],                 # tx_idx       
        int(log['blockNumber']),                 # blk_num
        timestamp,                               # timestamp
        log.args['from'],                        # frm
        log.args['to'],                          # to
        val,                                     # val
        format_decimals(val, str(decimals)),     # dec
    ]

def get_logs_df_wrk_unwrap(args) -> pd.DataFrame:
    return get_logs_df_wrk(*args)

def get_logs_df_wrk(
    addr: str, abi: any, dec: int, provider_url: str, frm: int, to: int,  retry: bool = True
) -> pd.DataFrame:
    df = pd.DataFrame(columns=[
        'tx_hash', 'tx_idx', 'blk_num', 'timestamp', 'frm', 'to', 'val','dec'
    ])
    addr = Web3.to_checksum_address(addr)
    provider = new_provider(provider_url)
    contract = provider.eth.contract(address=addr, abi=abi)
    logs = list()
    try: 
        logs = contract.events.Transfer().get_logs(fromBlock=frm, toBlock=to)
        for log in logs:
            df.loc[len(df.index)] = log_to_list(log, dec)
    except Exception as err:
        if retry:
            print('({}, {}) {} will retry'.format(frm, to, err))
            time.sleep(1)
            return get_logs_df_wrk(addr, abi, dec, provider_url, frm, to, False)
        else:
            print('({}, {}) {} ERR !!!'.format(frm, to, err))
    return df


# tx_hash block_num timestamp frm to val dec #Ymd
def get_logs_df(
    coin_id: str, 
    pump_timestamp: int,
    days: int,
    provider_url: str
) -> pd.DataFrame:
    addr, dec = get_address_and_decimals(coin_id)
    if addr is None or dec is None:
        return None
    
    provider = new_provider(provider_url)
    pump_block_num = block_num_for_timestamp(pump_timestamp, provider)
    st_block = pump_block_num - days * 24 * 60 * 5
    ranges = ranges_list(st_block, pump_block_num + (5 * 24 * 60 * 5), 250)
    print('({}, {})'.format(st_block, ranges[len(ranges) - 1][1]))
    abi = read_str_file('IERC20.json')
    df = pd.DataFrame(columns=[
        'tx_hash', 'tx_idx', 'blk_num', 'timestamp', 'frm', 'to', 'val','dec'
    ])
    with Pool(processes=cpu_count()) as p:
        vals = map(lambda x: (addr, abi, dec, provider_url, x[0], x[1]), ranges)
        vals = list(vals)
        dfs = p.map(get_logs_df_wrk_unwrap, vals)
    for batch_df in dfs:
        for index, row in batch_df.iterrows():
            df.loc[len(df.index)] = row
    return df


def timestamp_for_block_num_wrk_unwrap(args):
    return timestamp_for_block_num_wrk(*args)


def timestamp_for_block_num_wrk(block_num: int, provider_url: str) -> int:
    return new_provider(provider_url).eth.get_block(block_num)['timestamp']


def timestamp_for_block_num(df: pd.DataFrame, provider_url: str) -> pd.DataFrame:
    df = df.copy()    
    with Pool(processes=cpu_count()) as p:
        args = list(map(lambda x: (x, provider_url), df['blk_num']))
        results = p.map(timestamp_for_block_num_wrk_unwrap, args)
        df['timestamp'] = results
        df['Ymd'] = list(map(lambda x: year_month_day_str(x), df['timestamp']))
    return df


start = time.time()
df = eth_df_candles_pumps[coin_id]
print(coin_id, year_month_day_str(timestamp))
df_logs = get_logs_df(coin_id, timestamp, 90, provider_url)
print("seconds", time.time() - start, df_logs.shape)

# tx_hash block_num timestamp frm to val dec Ymd 
df_logs_timestamp = timestamp_for_block_num(df_logs, provider_url)

spurdo 2023-08-26
(17347139, 18031139)
(17400889, 17401139) {'code': -32700, 'message': 'parse error'} will retry


In [ ]:
def flatten_logs_to_tx(coin_id: str, df: pd.DataFrame, provider: web3.Web3) -> pd.DataFrame:
    df = df.copy()
    df.drop_duplicates('tx_hash', inplace = True)
    print(df.shape)
    df.insert(df.shape[1], 'method', np.arange(0, df.shape[0]))
    for i in range(len(df.index)):
        idx = df.index[i]
        tx_hash = df.loc[idx, 'tx_hash']
        tx_idx = int(df.loc[idx, 'tx_idx'])
        blk_num = int(df.loc[idx, 'blk_num'])
        print_progress('Fetching {}'.format(tx_hash), i, df.shape[0], True)
        tx = provider.eth.get_transaction_by_block(int(blk_num), int(tx_idx))
        input = provider.to_hex(tx.input) 
        method = input[:10] 
        df.loc[idx, 'frm'] = tx['from']
        if method == '0xa9059cbb':
            df.loc[idx, 'method'] = 'transfer'
            df.loc[idx, 'to'] = "0x" + input[34:74] 
        else:
            df.loc[idx, 'method'] = 'unknown'
    
    return df

df_tx = df_logs_timestamp.copy()
df_tx = flatten_logs_to_tx(coin_id, df_tx, provider)

In [ ]:
from math import ceil


# Ymd tx_cnt, tx_val_total, eoa_tx_cnt, eoa_val_total
def grouped_by_day(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    grouped = df.groupby('Ymd')
    res_df = pd.DataFrame(columns=['Ymd', 'txs_cnt', 'txs_val', 'eoa_txs_cnt', 'eoa_txs_val'])
    print("groups", len(grouped))
    for key, group in grouped:
        eoa_txs = group[group['method'] == 'transfer']
        eoa_txs_cnt = len(eoa_txs)
        eoa_txs_val = ceil(eoa_txs['dec'].sum())
        row = [key, len(group), ceil(group['dec'].sum()), eoa_txs_cnt, eoa_txs_val]
        res_df.loc[len(res_df.index)] = row
    
    res_df['tx_cnt_pct'] =  res_df['eoa_txs_cnt'] / res_df['txs_cnt']
    res_df['tx_val_pct'] =  res_df['eoa_txs_val'] / res_df['txs_val']
    res_df['high_swap'] =  res_df['tx_val_pct'] < 0.5 # res_df['eoa_txs_val'] / res_df['txs_val']
    res_df = res_df.round({'tx_cnt_pct': 4, 'tx_val_pct': 4})
    return res_df


pd.set_option('display.width', 256)
pd.set_option('display.max_colwidth', 24)

# Ymd tx_cnt, tx_val_total, eoa_tx_cnt, eoa_val_total
res_df = grouped_by_day(df_tx)
print(res_df.to_string())

pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

In [ ]:
# generate files for all
import os

pump_ratios = [2.0, 1.5, 4.0, 8.0]
min_volumes = [500000, 1000000, 10000000, 100000]

for ratio in pump_ratios:
    for vol in min_volumes:
        path = 'results/' + str(ratio).replace('.', '_') + '_' + str(vol)
        if not os.path.exists(path):
            os.mkdir(path)
        df_candles_pumps: Dict[str, pd.DataFrame] = dict()

        # for each chart
        for idx, key in enumerate(daily_candles):
            candles = daily_candles[key]
            print_progress("Searching for pumps", idx, len(daily_candles), True)
            if key not in df_charts: 
                continue    
            df_chart = df_charts[key]
            df_info = daily_candle_pumps_info(candles, pump_ratio, min_volume, df_chart)
            if not df_info.empty:
                df_candles_pumps[key] = df_info

        eth_df_candles_pumps = dict()
        for key, df in df_candles_pumps.items():
            address, decimals = get_address_and_decimals(key)
            if address is None or decimals is None:
                continue
            eth_df_candles_pumps[key] = df

        for coin_id, df in df_candles_pumps.items():
            for i in range(len(df.index)):
                timestamp = df.index[i]
                df_logs = get_logs_df(coin_id, timestamp, 90, provider_url)
                df_logs = timestamp_for_block_num(df_logs, provider_url)
                if df_logs is None:
                    continue
                df_tx = flatten_logs_to_tx(coin_id, df_logs, provider)
                res_df = grouped_by_day(df_tx)
                res_df.to_pickle(path + '/' + coin_id + '_' + str(i) + year_month_day_str(timestamp) + '.plk')    
# 
# 
#                 
#                 
# 
